# CTP using Gibb's sampling

LDA deel op basis van uitleg in http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

Using Gensim functionality

Dus we mogen ervan uitgaan dat we een dictionary hebben en bow vectors voor de documenten.
Het een en ander aan counts komt dan uit gensim. Maar hoe?

Wat hebben we nodig (LDA deel)?

* The number of words assigned to topic k in document d: $$n_{d;k}$$
* The number of times word w is assigned to topic k: $$n_{k;w}$$
* The total number of times any word is assigned to topic k: $$n_k$$
* Array z which will contain the current topic assignment for each of the N words in the corpus.

We beginnen met 1 extra perspectief (misschien perspectief-klasse?)

In [132]:
# Generate documents
import numpy as np

length_topic = 50
length_opinion = 20
num_topics = 3
topic_vocabulary = np.array(['zon', 'ijs', 'strand', 'vanille', 'chocola', 'broccoli', 'wortel'])
opinion_vocabulary = np.array(['warm', 'zwemmen', 'zonnig', 'bewolkt', 'vies', 'lekker', 'koud'])

real_theta_topic = np.array([[1.0, 0.0, 0.0],
                             [0.0, 1.0, 0.0],
                             [0.0, 0.0, 1.0],
                             [0.5, 0.5, 0.0],
                             [0.0, 0.5, 0.5]])
real_phi_topic = np.array([[0.4, 0.2, 0.4, 0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.3, 0.0, 0.35, 0.35, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5]])
real_phi_opinion = np.array([[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                             [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
                             [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]])

In [133]:
# generate the corpus
from collections import Counter

corpus = []

for m, tm in enumerate(real_theta_topic):
    doc = {'topic': [], 'opinion': []}
    topic_counter = Counter()
    for i in range(length_topic):
        # topic words
        topic = np.random.multinomial(1, tm)
        topic_index = np.where(topic==1)[0][0]
        topic_counter[topic_index] += 1
        word = np.random.multinomial(1, real_phi_topic[topic_index])
        doc['topic'].append(topic_vocabulary[np.where(word==1)[0][0]])
    
    #print topic_counter
    
    # select opinion (index) based on topic occurrence
    om = np.array([float(topic_counter[i]) for i in range(num_topics)])
    #print om
    # normalize
    om /= sum(om)
    for i in range(length_opinion):
        # opinion words
        topic = np.random.multinomial(1, om)
        topic_index = np.where(topic==1)[0][0]
        #print topic_index, real_phi_opinion[topic_index]
        word = np.random.multinomial(1, real_phi_opinion[topic_index])
        #print opinion_vocabulary[np.where(word==1)[0][0]]
        doc['opinion'].append(opinion_vocabulary[np.where(word==1)[0][0]])
        
    corpus.append(doc)
    
for i, doc in enumerate(corpus):
    print 'document', i+1, doc

document 1 {'topic': ['zon', 'zon', 'zon', 'zon', 'strand', 'strand', 'zon', 'ijs', 'strand', 'zon', 'strand', 'zon', 'ijs', 'zon', 'zon', 'zon', 'zon', 'strand', 'zon', 'strand', 'zon', 'zon', 'ijs', 'ijs', 'zon', 'strand', 'zon', 'zon', 'zon', 'zon', 'strand', 'zon', 'zon', 'ijs', 'zon', 'ijs', 'strand', 'zon', 'zon', 'strand', 'zon', 'strand', 'strand', 'strand', 'strand', 'strand', 'strand', 'strand', 'strand', 'ijs'], 'opinion': ['warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm', 'warm']}
document 2 {'topic': ['ijs', 'vanille', 'ijs', 'vanille', 'vanille', 'vanille', 'chocola', 'ijs', 'ijs', 'chocola', 'vanille', 'chocola', 'ijs', 'chocola', 'chocola', 'chocola', 'vanille', 'chocola', 'chocola', 'vanille', 'chocola', 'chocola', 'ijs', 'chocola', 'chocola', 'chocola', 'vanille', 'vanille', 'ijs', 'vanille', 'chocola', 'vanille', 'vanille', 'ijs', 'vanille', 'vanille', 'vanille', 'ij

In [31]:
from gensim import corpora

d = corpora.Dictionary(corpus)

In [134]:
# initialize z and o
import random
from collections import Counter

# topic
z = {}
ndk = np.zeros((len(corpus), num_topics), dtype=np.int)
nkw = np.zeros((num_topics, len(topic_vocabulary)), dtype=np.int)
nk = Counter()

# opinion
x = {}
nsd = np.zeros((len(corpus), num_topics), dtype=np.int)
nrs = np.zeros((num_topics, len(opinion_vocabulary)), dtype=np.int)
ntd = np.zeros(len(corpus), dtype=np.int)
ns = Counter()

for d, doc in enumerate(corpus):
    for i, word in enumerate(doc['topic']):
        t = random.randint(0,2)
        z[(d, i)] = t
        ndk[d][t] += 1
        word_index = np.where(topic_vocabulary==word)[0][0]
        nkw[t][word_index] += 1
        nk[t] += 1
        ntd[d] += 1
    for i, word in enumerate(doc['opinion']):
        o = random.randint(0,2)
        x[(d, i)] = o
        nsd[d][o] += 1
        word_index = np.where(opinion_vocabulary==word)[0][0]
        nrs[o][word_index] += 1
        ns[o] += 1

#print z
#print ndk
#print np.sum(ndk)
#print nkw
#print np.sum(nkw)
#print nk
print x
print nsd
print nrs
print ntd
print nk, ns

{(1, 17): 0, (0, 7): 0, (1, 6): 2, (0, 10): 0, (3, 7): 2, (2, 5): 2, (1, 11): 2, (4, 0): 1, (4, 19): 1, (0, 17): 1, (0, 4): 0, (1, 1): 1, (4, 10): 1, (3, 2): 2, (2, 6): 0, (4, 5): 0, (4, 16): 0, (2, 17): 0, (0, 1): 1, (3, 12): 0, (1, 12): 2, (4, 15): 2, (3, 1): 1, (2, 11): 2, (3, 18): 1, (2, 18): 0, (0, 14): 1, (3, 11): 1, (2, 1): 2, (1, 15): 2, (4, 12): 0, (2, 12): 0, (3, 17): 0, (1, 16): 1, (1, 5): 2, (0, 11): 2, (3, 6): 1, (2, 2): 1, (1, 10): 1, (4, 1): 0, (0, 18): 1, (1, 19): 1, (0, 5): 0, (1, 0): 2, (0, 8): 2, (4, 11): 2, (3, 5): 2, (2, 7): 0, (4, 6): 1, (4, 17): 2, (0, 2): 0, (3, 15): 0, (1, 3): 1, (4, 8): 0, (3, 0): 2, (2, 8): 0, (2, 19): 1, (0, 15): 1, (3, 10): 1, (1, 14): 2, (4, 13): 1, (2, 13): 0, (3, 16): 2, (1, 4): 1, (0, 12): 2, (3, 9): 1, (2, 3): 0, (1, 9): 0, (4, 2): 1, (2, 14): 0, (0, 19): 0, (1, 18): 0, (0, 6): 0, (1, 7): 0, (0, 9): 1, (3, 4): 0, (2, 4): 2, (4, 7): 0, (4, 18): 2, (0, 16): 1, (0, 3): 2, (3, 14): 1, (1, 2): 2, (4, 9): 2, (3, 3): 0, (2, 9): 0, (4, 4): 1, 

In [135]:
# iterate
#np.seterr(all='raise')

def p_z(alpha, beta, num_topics, num_words, d, topic, word_index):
    #print alpha, beta, num_topics, num_words, d, topic, word_index
    #print ndk[d][topic], nkw[topic][word_index], nk[topic]
    
    # De formule voor p_z is anders in het uitgewerkte voorbeeld dan in paper 'Finding scientific topics' en in CTP paper 
    # die hier staat is die uit het uitgewerkte voorbeeld
    return (ndk[d][topic]+alpha)*(nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)

def p_z2(alpha, beta, num_topics, num_words, d, topic, word_index):
    return ((ndk[d][topic]+alpha)/(np.sum(ndk[d])+num_topics*alpha)) * \
           (nkw[topic][word_index]+beta)/(nk[topic]+beta*num_words)


def p_x(beta_o, num_opinion_words, d, opinion, word_index):
    #print beta_o, num_opinion_words, d, opinion, word_index
    #print nrs[opinion][word_index], nsd[d][opinion], ntd[d]
    left = (nrs[opinion][word_index]+beta_o)/ns[o]+beta_o*num_opinion_words
    right = (float(nsd[d][opinion])/ntd[d])
    return left*right

def normalize(p):
    minimum = np.min(p)
    maximum = np.max(p)
    
    if minimum < 0:
        print '<0'
        return normalize((p - minimum)/(maximum - minimum))
    return p/sum(p)

num_iter = 100

alpha = 0.1
beta = beta_o = 0.02

theta = np.zeros((num_iter,  len(corpus), num_topics), dtype=np.float_)
phi = np.zeros((num_iter, num_topics, len(topic_vocabulary)), dtype=np.float_)
phi_o =np.zeros((num_iter, num_topics, len(opinion_vocabulary)), dtype=np.float_) 

for t in range(num_iter):
    for d, doc in enumerate(corpus):
        # Topic words
        for i, w in enumerate(doc['topic']):
            word = (d, i)
            topic = z[word]
            
            word_index = np.where(topic_vocabulary==w)[0][0]
            
            #if ndk[d][topic] > 0 and nkw[topic][word_index] > 0 and nk[topic] > 0:
            if True:
                ndk[d][topic] -= 1
                nkw[topic][word_index] -= 1
                nk[topic] -= 1
            
                p = [p_z2(alpha, beta, len(real_phi_topic), len(topic_vocabulary), d, j, word_index) for j in range(num_topics)]
                #print p
                # normalize
                p = normalize(p)
                #print p, sum(p)
            
                to = np.random.multinomial(1, p)
                topic = np.where(to==1)[0][0]
                
                z[word] = topic
            
                ndk[d][topic] += 1
                nkw[topic][word_index] += 1
                nk[topic] += 1

        # Opinion words
        for i, w in enumerate(doc['opinion']):
            word = (d, i)
            opinion = x[word]
            
            word_index = np.where(opinion_vocabulary==w)[0][0]
            
            nrs[opinion][word_index] -= 1
            ns[opinion] -= 1
            
            p = np.array([p_x(beta_o, len(opinion_vocabulary), d, j, word_index) for j in range(num_topics)], dtype=np.float)
            #print p, 
            p = normalize(p)
            #print p
            opinion = np.random.multinomial(1, p).argmax()
            #opinion = np.where(op==1)[0][0]
            
            x[word] = opinion
            
            nrs[opinion][word_index] += 1
            ns[opinion] += 1
        
    # calculate theta and phi
    theta[t] = (ndk+float(alpha))/(np.sum(ndk, axis=1, keepdims=True)+num_topics*alpha)
    phi[t] = (nkw+float(beta))/(np.sum(nkw, axis=1, keepdims=True)+len(topic_vocabulary)*beta)    
    phi_o[t] = (nrs+float(beta_o))/(np.sum(nrs, axis=1, keepdims=True)+len(opinion_vocabulary)*beta_o)
            #print 'new topic', topic
print ndk
print nkw
print nk
print nrs
print ns

[[ 0 50  0]
 [ 0  0 50]
 [50  0  0]
 [ 0 28 22]
 [25  0 25]]
[[ 0  0  0  0  0 34 41]
 [39  7 32  0  0  0  0]
 [ 0 32  0 32 33  0  0]]
Counter({2: 97, 1: 78, 0: 75})
[[ 8  0  0  0 22  0  4]
 [20  0  0  0  5  0 16]
 [ 8  0  0  0  6  0 11]]
Counter({1: 41, 0: 34, 2: 25})


In [136]:
print 'theta'
print 'found'
print np.mean(theta, axis=0)
print 'real'
print real_theta_topic
print
print 'phi'
print 'found'
print np.mean(phi, axis=0)
print 'real'
print real_phi_topic
print
print 'topics found:'
indexes = np.mean(phi, axis=0) > 0.01
for index in indexes:
    print topic_vocabulary[index]
print
print 'topics real'
indexes = real_phi_topic > 0.01
for index in indexes:
    print topic_vocabulary[index]
print 'opinions found:'
indexes = np.mean(phi_o, axis=0) > 0.01
for index in indexes:
    print opinion_vocabulary[index]
print
print 'opinions real'
indexes = real_phi_opinion > 0.01
for index in indexes:
    print opinion_vocabulary[index]

theta
found
[[ 0.00636183  0.98687873  0.00675944]
 [ 0.00516899  0.00536779  0.98946322]
 [ 0.99085487  0.00497018  0.00417495]
 [ 0.00616302  0.60139165  0.39244533]
 [ 0.50377734  0.00417495  0.49204771]]
real
[[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.   0.   1. ]
 [ 0.5  0.5  0. ]
 [ 0.   0.5  0.5]]

phi
found
[[  1.72127246e-03   3.50706640e-03   1.34082734e-03   2.87370114e-03
    7.40168713e-04   4.48448663e-01   5.41368301e-01]
 [  4.83452813e-01   1.13835389e-01   3.96205823e-01   1.30050789e-03
    2.33828093e-03   9.01253164e-04   1.96593243e-03]
 [  1.15924461e-03   3.12669164e-01   1.75792541e-03   3.34902380e-01
    3.47256113e-01   1.90049741e-03   3.54675400e-04]]
real
[[ 0.4   0.2   0.4   0.    0.    0.    0.  ]
 [ 0.    0.3   0.    0.35  0.35  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.5   0.5 ]]

topics found:
['broccoli' 'wortel']
['zon' 'ijs' 'strand']
['ijs' 'vanille' 'chocola']

topics real
['zon' 'ijs' 'strand']
['ijs' 'vanille' 'chocola']
['broccoli' 'wortel

In [137]:
def print_topic(weights, words):
    l = zip(words, weights)
    l.sort(key=lambda tup: tup[1])
    print l[::-1]

for t in np.mean(phi_o, axis=0):
    print_topic(t, opinion_vocabulary)

[('vies', 0.60959622024655724), ('warm', 0.20346120880129742), ('koud', 0.18479517528972964), ('lekker', 0.00053684891560397136), ('bewolkt', 0.00053684891560397136), ('zonnig', 0.00053684891560397136), ('zwemmen', 0.00053684891560397136)]
[('warm', 0.60491418395543772), ('koud', 0.26384515907140194), ('vies', 0.12905411367133415), ('lekker', 0.00054663582545651918), ('bewolkt', 0.00054663582545651918), ('zonnig', 0.00054663582545651918), ('zwemmen', 0.00054663582545651918)]
[('koud', 0.55129695247780464), ('warm', 0.23716350282730184), ('vies', 0.20786729383345592), ('lekker', 0.00091806271535943595), ('bewolkt', 0.00091806271535943595), ('zonnig', 0.00091806271535943595), ('zwemmen', 0.00091806271535943595)]


Is het toeval dat de indices van de topic topics en de opinion topics overeen komen of niet? En hoe kun je daarachter komen?